# QDC 2025 Challenge - Track B <br> QOBLIB - Maximum Independent Set Problem

In this challenge notebook, you'll learn how to tackle complex combinatorial optimizations using quantum algorithms. Specifically, we'll explore the Maximum Independent Set problem where we aim to find the largest set of vertices in a graph such that no two vertices in the set are adjacent (connected by an edge). This problem has numerous real-world applications including:

- **Network security**: Finding non-interfering communication channels
- **Scheduling**: Assigning non-conflicting tasks or resources
- **Molecular biology**: Analyzing protein interaction networks
- **Social networks**: Identifying independent influential nodes

This notebook is divided into three parts:
- Part I: learn the background of the Maximum Indepdent Set problem and how to fetch and load problem instances from the [Quantum Optimization Benchmarking Library (QOBLIB) repository](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library)
- Part II: You’ll then convert the problem into a Quadratic Unconstrained Binary Optimization (QUBO) form using the brand new [Qiskit Optimization Mapper Addon](https://github.com/Qiskit/qiskit-addon-opt-mapper) to be solved on a quantum computer.
- Part III: See an example of solving the problem using a [Qiskit function](https://quantum.cloud.ibm.com/docs/en/guides/functions) — the [Q-CTRL Optimization Solver](https://quantum.cloud.ibm.com/docs/en/guides/q-ctrl-optimization-solver).
- Part IV: Take on the challenge! You'll tackle a complex instance of the Maximum Indepdent Set problem using any quantum methods in your arsenal — whether it’s a Qiskit function, a custom QAOA, or other advanced techniques you’ll encounter throughout the QDC.

## Table of Contents

- [Part I: Understanding and Loading the Maximum Independent Set Problem (6 points)](#part-I)
- [Part II: QUBO Formulation (6 points)](#part-II)
- [Part III: Solving a MIS Problem with Q-CTRL Optimization Solver (MIS) (0 points)](#part-III)
- [Part IV: Main Challenge - Tackling a Complex Maximum Indepdent Set Problem Instance (88 points)](#part-IV)

# Imports

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
import requests
from urllib.parse import urlparse
import tempfile
import os

# Qiskit imports
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import qaoa_ansatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2 as Estimator, SamplerV2 as Sampler

# Q-CTRL Optimization Solver
from qiskit_ibm_catalog import QiskitFunctionsCatalog

# Import grading functions
from qc_grader.challenges.qdc_2025.qdc25_lab8 import (
    submit_name,
    grade_qctrl_function,
    grade_lab8_ex1,
    grade_lab8_ex2,
    grade_lab8_ex3,
)

from qiskit_addon_opt_mapper.applications import IndependentSet
from qiskit_addon_opt_mapper.converters import OptimizationProblemToQubo

# Sympy for polynomial representations
from sympy import symbols, Poly, srepr

print("All packages imported successfully!")

## Submit Team Name

In [ ]:
# Please use the exact spelling of your team name across teammates and across challenges

team_name = # Add team name string
submit_name(team_name)

<a id="part-I"></a>
# Part I: Understanding and Loading the Maximum Independent Set Problem (6 points)

The Maximum Independent Set (MIS) problem asks: given a graph, what is the largest set of vertices such that no two vertices in the set are connected by an edge?

## Problem Formulation

For a graph $G = (V, E)$ with $n$ vertices, we want to find the largest subset $S \subseteq V$ such that for any two vertices $u, v \in S$, there is no edge $(u,v) \in E$.

This can be formulated as an optimization problem:
- **Maximize**: $\sum_{i=1}^{n} x_i$ 
- **Subject to**: $x_i + x_j \leq 1$ for all edges $(i,j) \in E$
- Where $x_i \in \{0,1\}$ indicates whether vertex $i$ is selected

### Quantum Formulation

To use QAOA, we convert this to an unconstrained problem by adding penalty terms:
- **Minimize**: $-\sum_{i=1}^{n} x_i + P \sum_{(i,j) \in E} x_i x_j$
- Where $P$ is a large penalty weight

This becomes a Hamiltonian: $H = -\sum_{i=1}^{n} Z_i + P \sum_{(i,j) \in E} Z_i Z_j$

*Note*: This exercise demonstrates the core concepts but may require some QPU credits. Feel free to skip to Exercise 3 if you want to conserve resources for the main challenge.

## Data Loading and Problem Format

<div class="alert alert-block alert-success">
    
<b>Exercise 1: Complete the `parse_gph_file` Function (6 points)</b>

Your task is to complete the parsing logic that extrcts data from each line of the `.gph` file.

**What you need to do:**
- Read the file line by line
- Skip comments lines
- Create a graph based on information of the line with this format `p edge n m` which defines a graph with `n` vertices and `m` edges
- Lines starting with `e u v` define an edge between vertices `u` and `v`

</div>

In [ ]:
# Parse GPH File Format
import networkx as nx
def parse_gph_file(filepath):
    """
    Parse a .gph file (DIMACS graph format) into a NetworkX graph.

    The .gph format specifications:
    - Lines starting with 'c' are comments
    - Line starting with 'p edge n m' defines a graph with n vertices and m edges
    - Lines starting with 'e u v' define an edge between vertices u and v

    Args:
        filepath (str): Path to the .gph file

    Returns:
        nx.Graph: Parsed graph
    """
    graph = nx.Graph()

    if not filepath or not os.path.exists(filepath):
        print("File not found. Please check the path or download manually.")
        return None

    with open(filepath, 'r') as f:
        lines = f.readlines()

    print(f"Parsing {len(lines)} lines...")

    for line_num, line in enumerate(lines, 1):
        line = line.strip()

        if not line or line.startswith('c'):
            # Skip comments and empty lines
            continue

        elif line.startswith('p edge'):
            # Parse problem definition: p edge <vertices> <edges>
            parts = line.split()
            if len(parts) >= 4:
                
                ### Don't change any code before this line ###
                

                
                ### Don't change any code past this line ###
        
        elif line.startswith('e'):
            # Parse edge: e <vertex1> <vertex2>
            parts = line.split()
            if len(parts) >= 3:
                try:
                    
                    ### Don't change any code before this line ###
                    

                    
                    ### Don't change any code before this line ###
                
                except ValueError:
                    print(f"Warning: Could not parse edge on line {line_num}: {line}")
        else:
            print(f"Warning: Unknown line format on line {line_num}: {line}")

    print(f"Successfully parsed graph:")
    print(f"   - Vertices: {len(graph.nodes)}")
    print(f"   - Edges: {len(graph.edges)}")
    print(f"   - Density: {nx.density(graph):.3f}")

    return graph

In [ ]:
# Submit your answer using following code

grade_lab8_ex1(parse_gph_file) # Expected result type: Callable

In [ ]:
# Download QOBLIB Instance

import requests
import os
import tempfile

qoblib_url = "https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/raw/main/07-independentset/instances/es60fst02.gph"

def download_qoblib_instance(url, filename=None):
    """
    Download a QOBLIB instance from the given URL.

    Args:
        url (str): URL to the .gph file
        filename (str, optional): Local filename to save to

    Returns:
        str: Path to the downloaded file
    """
    if filename is None:
        # Extract filename from URL
        filename = url.split('/')[-1].split('?')[0]

    print(f"Downloading {filename} from QOBLIB...")

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Save to the same directory as the notebook
        current_dir = os.getcwd()
        filepath = os.path.join(current_dir, filename)

        with open(filepath, 'wb') as f:
            f.write(response.content)

        print(f"Downloaded successfully to: {filepath}")
        print(f"File size: {len(response.content)} bytes")
        return filepath

    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")
        print("Alternative: You can manually download from:")
        print(f"   {url}")
        print("   and place it in the same directory as this notebook")
        return None

downloaded_file = download_qoblib_instance(qoblib_url)

In [ ]:
# Parse the downloaded file
if downloaded_file:
    qoblib_graph = parse_gph_file(downloaded_file)
else:
    print("Attempting to load from current directory...")
    # Try to load from current directory if download failed
    local_file = "es60fst02.gph"
    if os.path.exists(local_file):
        qoblib_graph = parse_gph_file(local_file)
    else:
        print("Please download the file manually from the URL above")
        qoblib_graph = None

<a id="part-II"></a>
# Part II: QUBO Formulation (6 points)

In [ ]:
# Use Qiskit Addon Opt Mapper to generate the MIS Hamiltonian
def create_mis_graph(n_vertices=6, edge_probability=0.4, seed=42):
    """Create a random graph for the MIS problem."""
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(n_vertices, edge_probability, seed=seed)
    return G

def visualize_mis_solution(graph, solution_bitstring):
    """Visualize the MIS solution on the graph."""
    pos = nx.spring_layout(graph, seed=42)
    node_colors = ['red' if solution_bitstring[i] == '1' else 'lightblue'
                   for i in range(len(graph.nodes))]

    plt.figure(figsize=(8, 6))
    nx.draw(graph, pos, node_color=node_colors, with_labels=True,
            node_size=500, font_size=16, font_weight='bold')
    plt.title("Maximum Independent Set Solution\n(Red = Selected vertices)")
    plt.show()

# Create a small example graph
mis_graph = create_mis_graph(n_vertices=6, edge_probability=0.3)
print(f"Created graph with {len(mis_graph.nodes)} vertices and {len(mis_graph.edges)} edges")

# Visualize the problem graph
pos = nx.spring_layout(mis_graph, seed=42)
plt.figure(figsize=(8, 6))
nx.draw(mis_graph, pos, with_labels=True, node_color='lightblue',
        node_size=500, font_size=16, font_weight='bold')
plt.title("MIS Problem Graph")
plt.show()

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 2: Convert to QUBO (6 points)</b>

Convert the Maximum Independent Set problem to QUBO format using the [`qiskit_addon_opt_mapper`](https://github.com/Qiskit/qiskit-addon-opt-mapper) package.

**Tasks:**
1. Create an `IndepdentSet` object with the graph.
2. Convert the problem to QUBO format with penalty parameter = 2

**Note**: Please use the small graph `mis_graph` generated by `create_mis_graph` function for this exercise. For the final challenge, please use the `es60fst02` instance.
</div>

In [ ]:
# Convert to Hamiltonian using Qiskit Addon Opt Mapper
### Write your code below here ###

mis = 
qubo = 
mis_hamiltonian = 

### Don't change any code past this line ###
print(f"Hamiltonian generated with {len(mis_hamiltonian.paulis)} terms")
print("Sample terms:", mis_hamiltonian.paulis[:3])

In [ ]:
# Submit your answer using following code

grade_lab8_ex2(qubo) # Expected result type: OptimizationProblem

<a id="part-III"></a>
# Part III: Solving a MIS Problem with Q-CTRL Optimization Solver (MIS) (0 points)

In this part, you'll solve the Maximum Independent Set (MIS) problem for a 100-node graph, following [IBM's QAOA tutorial, Part II](https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm#part-ii-scale-it-up), but using the Q-CTRL Optimization Solver.

We'll:

1. Build the 100-node graph from the IBM tutorial
2. Convert the MIS problem to a sympy polynomial using Q-CTRL's recommended approach
3. Submit the problem to the Q-CTRL Optimization Solver

*Note*: This exercise demonstrates the core concepts but would require some QPU credits (~20min). Feel free to skip to Exercise 3 if you want to conserve resources for the main challenge.

In [ ]:
# # If you haven't done so already
# # Uncomment the follow code to save the QDC 2025 account provided to you
#
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# QiskitRuntimeService.save_account(
#     channel='ibm_quantum_platform',
#     token=your_api_key,
#     instance=your_crn,
#     name="qdc-2025"
# )

In [ ]:
# Load the Qiskit Functions Catalog
catalog = QiskitFunctionsCatalog(name="qdc-2025")

# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
display(catalog.list())

print("Successfully initialized Qiskit Functions Catalog")
print("Ready to load quantum optimization functions")

<a id="function-exercise"></a>
<div class="alert alert-block alert-success">
    
<b>Function Exercise: Load the Q-CTRL Optimization Solver Function (0 points)</b>

Load the Q-CTRL Optimization Solver function from the catalog.

**Tasks:**
1. Determine the correct function name for the Q-CTRL Optimization Solver
2. Use `catalog.load()` to load the function
3. Store the result in variable `iskay_solver`

</div>

In [ ]:
### Write your code below here ###

function_name = ""  # TODO: Specify the Q-CTRL Optimization Solver
solver = catalog.load(function_name)

In [ ]:
# Submit your answer using following code

grade_qctrl_function(solver) # Expected result type: QiskitFunction

In [ ]:
# Q-CTRL Optimization Solver Demo for MIS (100-node example)
# QAOA Implementation for MIS (Optional - requires QPU credits)

# Uncomment and run this section if you want to execute on quantum hardware

"""
# Step 1: Create the 100-node MIS problem graph (from IBM QAOA tutorial Part II)
def create_100_node_mis_graph():
    # Create the 100-node graph from IBM's QAOA tutorial (Part II).
    # https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm#part-ii-scale-it-up
    import itertools
    G = nx.Graph()
    G.add_nodes_from(range(100))
    # Edges: connect i to i+1 and i+2 (modulo 100)
    for i in range(100):
        G.add_edge(i, (i + 1) % 100)
        G.add_edge(i, (i + 2) % 100)
    return G

large_mis_graph = create_100_node_mis_graph()
print(f"Created graph with {len(large_mis_graph.nodes)} vertices and {len(large_mis_graph.edges)} edges")

# Visualize the problem graph (optional, can be slow for 100 nodes)
# pos = nx.spring_layout(large_mis_graph, seed=42)
# plt.figure(figsize=(10, 8))
# nx.draw(large_mis_graph, pos, with_labels=True, node_color='lightblue', node_size=200, font_size=8)
# plt.title("100-node MIS Problem Graph (IBM QAOA Tutorial Part II)")
# plt.show()

# Step 2: Convert the MIS problem to a sympy.Poly for Q-CTRL solver
from sympy import symbols, Poly, srepr

def mis_to_cost_polynomial(graph, penalty_weight=None):
    # Convert a Maximum Independent Set problem to a cost polynomial for Q-CTRL solver.
    # Minimize: -∑ᵢ xᵢ + P ∑_{(i,j) ∈ E} xᵢxⱼ
    vertices = list(graph.nodes())
    n_vertices = len(vertices)
    vertex_mapping = {v: i for i, v in enumerate(vertices)}
    variables = symbols([f'n[{i}]' for i in range(n_vertices)])
    if penalty_weight is None:
        penalty_weight = n_vertices + 1
    cost = 0
    for vertex in vertices:
        idx = vertex_mapping[vertex]
        cost -= variables[idx]
    for u, v in graph.edges():
        idx_u = vertex_mapping[u]
        idx_v = vertex_mapping[v]
        cost += penalty_weight * variables[idx_u] * variables[idx_v]
    polynomial = Poly(cost, *variables)
    return polynomial, vertex_mapping

large_mis_poly, large_vertex_mapping = mis_to_cost_polynomial(large_mis_graph)
print("MIS cost polynomial:")
print(large_mis_poly)

# Run Q-CTRL Optimization Solver (requires credentials and QPU credits)
cost_string = srepr(large_mis_poly)

print("Submitting 100-node MIS problem to Q-CTRL Optimization Solver...")
job = solver.run(
    problem=cost_string,
    backend_name=backend_name
)
print(f"Job submitted! Job ID: {job.job_id}")
print(f"Status: {job.status()}")

# To retrieve results after completion:
# result = job.result()
# solution_bitstring = result['solution_bitstring']
# final_cost = result['solution_bitstring_cost']
# print(f"Solution: {solution_bitstring}")
# print(f"Independent set size: {solution_bitstring.count('1')}")
"""

<a id="part-IV"></a>
# Part IV: Main Challenge - Tackling a Complex Maximum Indepdent Set Problem Instance (88 points)

Welcome to the main challenge! You'll work with a real optimization benchmark from QOBLIB (Quantum Optimization Benchmark Library), a collection of hard optimization problems used to test quantum algorithms.

<div class="alert alert-block alert-success">

<b>Exercise 3: Solving the `es60fst02` instance (88 points)</b>

The `es60fst02` graph has 186 vertices which are more than the number of qubits (156 qubits) in the QPUs available for QDC. Therefore, you need to develop clever approximation strateiges.

These are steps
1. **Download** the QOBLIB instance
2. **Parse** the graph format
3. **Reduce** the problem size strategically
4. **Solve** using a quantum optimization algorithm such as Q-CTRL optimizer function
5. **Optimize** to get as close as possible to the true optimum

</div>

<div class="alert alert-block alert-warning">

The reference solution for this instance is available in QOBLIB, but your goal is to design a general, scalable approach capable of solving problems of similar complexity with similar performance.  

> ⚠️ **Important:** Solutions that directly use or rely on the known answer will be disqualified. We’re looking for genuine quantum innovation — creative, algorithmic approaches that push the limits of what’s possible!

After the submission deadline, top scorers will have their notebooks reviewed by an expert panel, who will verify the results by running additional problem instances.  
Please refer to the [QDC contest rules](https://github.com/qiskit-community/qdc-challenges-2025/contest_rules.md) for more details on evaluation and submission procedures.
</div>

<div class="alert alert-block alert-info">

You are free to unleash any quantum techniques in your arsenal, including (but not limited to):  
- Qiskit functions
- Custom QAOA implementations  
- Advanced transpilation or circuit optimization strategies  
- Alternative problem formulations
- Classical preprocessing or postprocessing tricks

See the [Quantum Optimization Best Practicies repository](https://github.com/qiskit-community/qopt-best-practices) for ideas.
</div>

In [ ]:
# MIS Utility Functions (adapted from Q-CTRL documentation)

def mis_to_cost_polynomial(graph, vertex_mapping=None):
    """
    Convert a Maximum Independent Set problem to a cost polynomial for Q-CTRL solver.

    The MIS problem is formulated as:
    Minimize: -∑ᵢ xᵢ

    Args:
        graph (nx.Graph): The input graph
        vertex_mapping (dict): Optional mapping from graph vertices to variable indices

    Returns:
        sympy.Poly: Polynomial representation of the MIS cost function
    """
    from sympy import symbols, Poly

    vertices = list(graph.nodes())
    n_vertices = len(vertices)

    # Create vertex to index mapping if not provided
    if vertex_mapping is None:
        vertex_mapping = {v: i for i, v in enumerate(vertices)}

    # Create symbolic variables
    variables = symbols([f'n[{i}]' for i in range(n_vertices)])

    # Calculate penalty weight: should be larger than maximum possible reward
    penalty_weight = n_vertices + 1

    # Initialize cost function
    cost = 0

    # Reward terms: -xᵢ for each vertex (we want to maximize number of selected vertices)
    for vertex in vertices:
        idx = vertex_mapping[vertex]
        cost -= variables[idx]

    # Penalty terms: P * xᵢ * xⱼ for each edge (i,j)
    for u, v in graph.edges():
        idx_u = vertex_mapping[u]
        idx_v = vertex_mapping[v]
        cost += penalty_weight * variables[idx_u] * variables[idx_v]

    # Convert to sympy Poly
    polynomial = Poly(cost, *variables)

    return polynomial, vertex_mapping

def check_independent_set(graph, bitstring, vertex_mapping):
    """
    Check if a bitstring represents a valid independent set.

    Args:
        graph (nx.Graph): The graph
        bitstring (str): Binary string representing selected vertices
        vertex_mapping (dict): Mapping from vertices to bitstring indices

    Returns:
        bool: True if the set is independent, False otherwise
    """
    # Get selected vertices
    selected_vertices = []
    reverse_mapping = {v: k for k, v in vertex_mapping.items()}

    for i, bit in enumerate(bitstring):
        if bit == '1':
            selected_vertices.append(reverse_mapping[i])

    # Check if any two selected vertices are adjacent
    for u in selected_vertices:
        for v in selected_vertices:
            if u != v and graph.has_edge(u, v):
                return False

    return True

def maximum_independent_set_size(bitstring):
    """Calculate the size of the independent set represented by the bitstring."""
    return bitstring.count('1')

def analyze_graph_structure(graph):
    """Analyze key properties of the graph to inform reduction strategies."""
    print("Graph Analysis:")
    print(f"   Vertices: {len(graph.nodes)}")
    print(f"   Edges: {len(graph.edges)}")
    print(f"   Density: {nx.density(graph):.4f}")
    print(f"   Average degree: {sum(dict(graph.degree()).values()) / len(graph.nodes):.2f}")

    # Compute degree statistics
    degrees = [d for n, d in graph.degree()]
    print(f"   Degree range: {min(degrees)} - {max(degrees)}")

    # Find connected components
    components = list(nx.connected_components(graph))
    print(f"   Connected components: {len(components)}")
    if len(components) > 1:
        component_sizes = [len(c) for c in components]
        print(f"   Component sizes: {sorted(component_sizes, reverse=True)}")

    return {
        'vertices': len(graph.nodes),
        'edges': len(graph.edges),
        'density': nx.density(graph),
        'avg_degree': sum(degrees) / len(degrees),
        'max_degree': max(degrees),
        'min_degree': min(degrees),
        'components': len(components)
    }

# Analyze the QOBLIB graph if it was loaded successfully
if qoblib_graph is not None:
    graph_stats = analyze_graph_structure(qoblib_graph)

    # Visualize degree distribution
    degrees = [d for n, d in qoblib_graph.degree()]
    plt.figure(figsize=(10, 6))
    plt.hist(degrees, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Vertex Degree')
    plt.ylabel('Frequency')
    plt.title('Degree Distribution of es60fst02 Graph')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Graph not loaded - skipping analysis")

## The Challenge: Graph Reduction Strategies

The es60fst02 graph has **186 vertices**, which doesn't yet fit onto a typical Heron device. You need to develop clever strategies to reduce the problem size while preserving as much information as possible about the optimal solution.

### Strategy Ideas:

1. **Vertex Selection**:
   - High-degree vertices are less likely to be in an independent set
   - Low-degree vertices might be good candidates

2. **Subgraph Extraction**:
   - Solve multiple smaller subproblems
   - Combine solutions from different subgraphs

4. **Randomized Methods**:
   - Random sampling with multiple runs
   - Monte Carlo approaches

<div class="alert alert-block alert-info">
<b>Tips for Success:</b>

**Try Multiple Strategies**: Test different graph reduction approaches  
**Iterate and Refine**: Use insights from initial runs to improve your approach  
**Document Your Process**: Clear explanations of your strategy will help the validation process  
**Validate Results**: Always check that your solution is a valid independent set

In [ ]:
# YOUR TURN: Implement your own strategy!
def your_custom_strategy(graph, target_size=150):
    """
    YOUR CUSTOM STRATEGY: Implement your own approach here!

    Return: (subgraph, mapping) where mapping maps original->new vertex labels
    """
    # TODO: Implement your strategy here
    # This is a placeholder - replace with your implementation

    # Example: Random sampling (you can do better!)
    vertices = list(graph.nodes())
    np.random.seed(42)
    selected_vertices = np.random.choice(vertices, min(target_size, len(vertices)), replace=False)

    subgraph = graph.subgraph(selected_vertices).copy()
    mapping = {old: new for new, old in enumerate(sorted(subgraph.nodes()))}
    subgraph = nx.relabel_nodes(subgraph, mapping)

    print(f"Your strategy: Selected {len(selected_vertices)} vertices")
    print(f"Subgraph has {len(subgraph.edges)} edges")

    return subgraph, mapping

# Test different strategies if graph is available
if qoblib_graph is not None:
    print("\nTesting reduction strategies:")
    print("=" * 50)

    # Test each strategy
    strategies = [
        ("Your custom strategy", your_custom_strategy)
    ]

    strategy_results = {}

    for name, strategy_func in strategies:
        print(f"\nTesting: {name}")
        try:
            reduced_graph, vertex_mapping = strategy_func(qoblib_graph, target_size=150)
            strategy_results[name] = (reduced_graph, vertex_mapping)
            print(f"   Success: {len(reduced_graph.nodes)} vertices, {len(reduced_graph.edges)} edges")
            print(f"   Density: {nx.density(reduced_graph):.4f}")
        except Exception as e:
            print(f"   Failed: {e}")
            strategy_results[name] = None
else:
    print("Original graph not available - implement strategies and test with your own graphs")

In [ ]:
# Solving the reduced graphs using quantum methods and find the solution of the original graph











solution_bitstring = 

<div class="alert alert-block alert-warning">
    
<b>Score Formula</b>

The scoring function is simply defined as:

$$\text{score} = \text{size of the indepdent set}$$

For this problem instance, the maximum indepdent set is `88` which is the highest possible score.

</div>

In [ ]:
# Submit your answer using following code

grade_lab8_ex3(solution_bitstring) # Expected result type: str

# References

1. [Q-CTRL Optimization Solver Documentation](https://docs.quantum.ibm.com/guides/q-ctrl-optimization-solver)
2. [IBM QAOA Tutorial](https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm)
3. [QOBLIB Benchmark Library](https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library)
4. [NetworkX Documentation](https://networkx.org/documentation/stable/)

# Additional Information

**Created by:** You Quan Chong, Yulun Wang

**Advised by:** Junye Huang

**Version:** 1.0.0

# Qiskit packages versions

In [ ]:
import qiskit
import qiskit_ibm_runtime
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')